In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [3]:
data = pd.read_csv(r"C:\Users\User\Downloads\AMDARI\reder-predictions\reder-predictions\dataset\Dataset.xlsx - Sheet1.csv")

In [4]:
data.columns

Index(['CustomerID', 'Name', 'Age', 'Gender', 'Location', 'Email', 'Phone',
       'Address', 'Segment', 'PurchaseHistory', 'SubscriptionDetails',
       'ServiceInteractions', 'PaymentHistory', 'WebsiteUsage',
       'ClickstreamData', 'EngagementMetrics', 'Feedback',
       'MarketingCommunication', 'NPS', 'ChurnLabel', 'Timestamp'],
      dtype='object')

In [5]:
cols_to_extract = ['PurchaseHistory', 'SubscriptionDetails',
       'ServiceInteractions', 'PaymentHistory', 'WebsiteUsage',
       'ClickstreamData', 'EngagementMetrics', 'Feedback',
       'MarketingCommunication']

In [6]:
# subsetting

data["PurchaseHistory"].iloc[0]

"[{'Product': 'Frozen Cocktail Mixes', 'Frequency': 8, 'Value': 884.43}, {'Product': 'Printer, Copier & Fax Machine Accessories', 'Frequency': 7, 'Value': 397.14}, {'Product': 'Hockey Stick Care', 'Frequency': 10, 'Value': 498.92}, {'Product': 'Guacamole', 'Frequency': 2, 'Value': 718.43}, {'Product': 'Mortisers', 'Frequency': 2, 'Value': 614.08}, {'Product': 'Rulers', 'Frequency': 6, 'Value': 221.68}, {'Product': 'Invitations', 'Frequency': 3, 'Value': 660.04}]"

In [7]:
data["SubscriptionDetails"].iloc[0]

"{'Plan': 'Express', 'Start_Date': '2020-06-08', 'End_Date': '2022-10-27'}"

1. Convert the string to Python data structure (list, dictionary)
2. Exoplode the list to dictionary.
3. Then convert dictionary to Python dataframe
4. Append to a dictionary

In [8]:
import ast

ast.literal_eval(data["SubscriptionDetails"].iloc[0])

{'Plan': 'Express', 'Start_Date': '2020-06-08', 'End_Date': '2022-10-27'}

In [9]:
# showing how the explode function works.
# Nb: explode does not work on lists. it only works for pandas series

data["PurchaseHistory"].apply(ast.literal_eval).explode()


0        {'Product': 'Frozen Cocktail Mixes', 'Frequenc...
0        {'Product': 'Printer, Copier & Fax Machine Acc...
0        {'Product': 'Hockey Stick Care', 'Frequency': ...
0        {'Product': 'Guacamole', 'Frequency': 2, 'Valu...
0        {'Product': 'Mortisers', 'Frequency': 2, 'Valu...
                               ...                        
12481    {'Product': 'Tractor Parts & Accessories', 'Fr...
12481    {'Product': 'Video Cards & Adapters', 'Frequen...
12481    {'Product': 'Clear Kerosene', 'Frequency': 3, ...
12481    {'Product': 'Tripod Spreaders', 'Frequency': 8...
12482    {'Product': 'Pool Brushes & Brooms', 'Frequenc...
Name: PurchaseHistory, Length: 68628, dtype: object

In [10]:
def convert_string(col: str) -> any:
    """ This function is used to convert string to list/dict using ast.literal_eval """
    if isinstance(col,str):
        return ast.literal_eval(col)
    return col

extracted_tables = {}

for col in cols_to_extract:
    data[col] = data[col].apply(convert_string)

    if isinstance(data[col].iloc[0], list):
        temp = data[['CustomerID', col]].explode(col)
        normalized = pd.json_normalize(temp[col])
        normalized['CustomerID'] = temp['CustomerID'].values
    elif isinstance(data[col].iloc[0], dict):
        normalized = pd.json_normalize(data[col])
        normalized['CustomerID'] = data['CustomerID'].values

    extracted_tables[col] = normalized

In [11]:
extracted_tables["PurchaseHistory"]

,Product,Frequency,Value,CustomerID
0,Frozen Cocktail Mixes,8,884.43,1001
1,"Printer, Copier & Fax Machine Accessories",7,397.14,1001
2,Hockey Stick Care,10,498.92,1001
3,Guacamole,2,718.43,1001
4,Mortisers,2,614.08,1001
...,...,...,...,...
68623,Tractor Parts & Accessories,9,845.64,13482
68624,Video Cards & Adapters,3,791.84,13482
68625,Clear Kerosene,3,908.41,13482
68626,Tripod Spreaders,8,238.25,13482


In [12]:
for key, value in extracted_tables.items():
    print(f'{key}: {value.shape}')
    # globals()[name] = df


PurchaseHistory: (68628, 4)
SubscriptionDetails: (12483, 4)
ServiceInteractions: (254253, 3)
PaymentHistory: (37449, 3)
WebsiteUsage: (12483, 3)
ClickstreamData: (319616, 4)
EngagementMetrics: (12483, 3)
Feedback: (12483, 3)
MarketingCommunication: (68762, 4)


### List Comprehension


In [ ]:
# names = ['Emma', 'Kenny', 'Bright', 'Ivy']

# # Method 1
# for name in names:
#     print(name)

# # Method 2
# [print(name) for name in names]

In [13]:
# Unpacking

PurchaseHistory, SubscriptionDetails, ServiceInteractions,\
    PaymentHistory,WebsiteUsage,ClickstreamData, EngagementMetrics,\
        Feedback, MarketingCommunication = [extracted_tables[col] for col in extracted_tables.keys()]

### Explanatory Data Analysis

### Exploratory Data Analysis

### Future Engineering

#### Purchase Behaviour Features

Idea: Customers who spend more and buy more are less likely to churn.

We calculate:
1. Total/average spending (loyalty indicator)
2. Purchase consistency (stable vs consistent buyers)
3. Product diversity (engaged vs single product users)

In [14]:
# PurchaseHistory.groupby('CustomerID').agg({
#     'Value': 'sum',
#     'Frequency': 'sum',
#     'Product': 'nunique'
# }).rename(columns = {
#     'Value': 'Total_Purchase_Value',
#     'Frequency': 'Total_Purchase_Frequency',
#     'Product': 'Unique_Product_Count'
# }).reset_index()

PurchaseHistory = PurchaseHistory.groupby('CustomerID').agg(
    Total_Purchase_Value=('Value', 'sum'),
    Total_Purchase_Frequency=('Frequency', 'sum'),
    # Products_Purchased=('Product', lambda x: ' | '.join(sorted(set(x.dropna().astype(str))))),
    Unique_Product_Count=('Product', 'nunique')
).reset_index()

In [15]:
PurchaseHistory

,CustomerID,Total_Purchase_Value,Total_Purchase_Frequency,Unique_Product_Count
0,1001,3994.72,38,7
1,1002,2844.35,4,3
2,1003,1866.52,14,3
3,1004,1378.64,28,5
4,1005,2425.05,39,6
...,...,...,...,...
12478,13479,1196.56,14,3
12479,13480,710.57,1,1
12480,13481,5154.42,63,10
12481,13482,6055.16,58,9


#### Subscription Lifecycle Features

Idea: Long-term subscribers with active accounts are less likely to churn.

We calculate:

1. How long they've been subscribed (tenure)
2. wether subscription is still active
3. Subscription plan type

In [16]:
# SubscriptionDetails['Start_Date'] = pd.to_datetime(SubscriptionDetails['Start_Date'], errors='coerce')

# SubscriptionDetails['End_Date'] = pd.to_datetime(SubscriptionDetails['End_Date'], errors='coerce')

# # SubscriptionDmost_recent_end = SubscriptionDetails['end_date'].dropna().max()

# SubscriptionDetails['Tenure_Days'] = (SubscriptionDetails['End_Date'] - SubscriptionDetails['Start_Date']).dt.days

# SubscriptionDetails



SubscriptionDetails[['Start_Date', 'End_Date']] = SubscriptionDetails[['Start_Date', 'End_Date']].apply(
    pd.to_datetime, errors='coerce'
)

SubscriptionDetails['subscription_duration_days'] = (SubscriptionDetails['End_Date'] - SubscriptionDetails['Start_Date']).dt.days
SubscriptionDetails['subscription_age_days'] = (pd.to_datetime('2023-01-01') - SubscriptionDetails['Start_Date']).dt.days
SubscriptionDetails['is_active'] = (SubscriptionDetails['End_Date'] > pd.to_datetime('2023-01-01')).astype(int)

SubscriptionDetails = SubscriptionDetails[['CustomerID', 'Plan', 'subscription_duration_days', 'subscription_age_days', 'is_active']]


In [17]:
SubscriptionDetails

,CustomerID,Plan,subscription_duration_days,subscription_age_days,is_active
0,1001,Express,871,937,0
1,1002,Pro,290,529,0
2,1003,Essential,319,1184,0
3,1004,Smart,803,1083,0
4,1005,Basic,580,633,0
...,...,...,...,...,...
12478,13479,Essential,745,1296,0
12479,13480,Flex,18,22,0
12480,13481,Deluxe,20,546,0
12481,13482,Gold,484,894,0


#### Service Interaction Patterns

Idea: Frequent support contacts (especially calls) may signal dissatisfaction

We calculate:

1. Total interactions
2. Last interaction date

In [18]:
# ServiceInteractions['Date'] = pd.to_datetime(ServiceInteractions['Date'])
ServiceInteractions = ServiceInteractions.groupby('CustomerID').agg(
    Last_Interaction_Date = ('Date', lambda x: pd.to_datetime(x.max())),
    Total_Interactions = ('Type', 'count')
).reset_index()

ServiceInteractions

,CustomerID,Last_Interaction_Date,Total_Interactions
0,1001,2021-07-25,4
1,1002,2022-12-13,19
2,1003,2022-01-04,3
3,1004,2022-11-10,59
4,1005,2022-12-19,10
...,...,...,...
12478,13479,2022-10-09,10
12479,13480,2022-11-05,3
12480,13481,2022-12-08,26
12481,13482,2022-11-15,13


#### Payment Reliability Features

Idea: Late payment indicate financial stress or dissatisfaction.

We calculate:

1. Total late payments
2. Late payment rate (percentage of payments that are late)
3. Payment risk score (combined metric)


In [19]:
PaymentHistory = PaymentHistory.groupby('CustomerID').agg(
    total_late_payments = ('Late_Payments', 'sum'),
    payment_count = ('Method', 'count')
).reset_index()
PaymentHistory['late_payment_rate'] = PaymentHistory['total_late_payments'] / (PaymentHistory['payment_count'] + 1)
PaymentHistory['payment_risk_score'] = PaymentHistory['total_late_payments'] * PaymentHistory['late_payment_rate']

PaymentHistory

,CustomerID,total_late_payments,payment_count,late_payment_rate,payment_risk_score
0,1001,40,3,10.00,400.00
1,1002,10,3,2.50,25.00
2,1003,8,3,2.00,16.00
3,1004,79,3,19.75,1560.25
4,1005,2,3,0.50,1.00
...,...,...,...,...,...
12478,13479,3,3,0.75,2.25
12479,13480,6,3,1.50,9.00
12480,13481,83,3,20.75,1722.25
12481,13482,67,3,16.75,1122.25


#### Digital Engagement Features

Idea: Active website users are more engaged and less likely to churn

we calculate:

1. Page views and time spent
2. Engagement ratio (time per page)
3. Engagement intensity (overall activity level)

In [20]:
WebsiteUsage

,PageViews,TimeSpent(minutes),CustomerID
0,49,15,1001
1,100,9,1002
2,1,97,1003
3,25,31,1004
4,77,51,1005
...,...,...,...
12478,70,57,13479
12479,71,66,13480
12480,96,1,13481
12481,63,2,13482


In [21]:
WebsiteUsage['engagement_ratio'] = WebsiteUsage['TimeSpent(minutes)'] / (WebsiteUsage['PageViews'] + 1)
WebsiteUsage['engagement_intensity'] = WebsiteUsage['PageViews'] * WebsiteUsage['TimeSpent(minutes)']

In [22]:
WebsiteUsage

,PageViews,TimeSpent(minutes),CustomerID,engagement_ratio,engagement_intensity
0,49,15,1001,0.300000,735
1,100,9,1002,0.089109,900
2,1,97,1003,48.500000,97
3,25,31,1004,1.192308,775
4,77,51,1005,0.653846,3927
...,...,...,...,...,...
12478,70,57,13479,0.802817,3990
12479,71,66,13480,0.916667,4686
12480,96,1,13481,0.010309,96
12481,63,2,13482,0.031250,126


#### Clickstream Behaviour Features

Idea: Specific click actions (adding to cart, searching) show purchase intent. 
We calculate:

1. Total actions taken
2. Cart conversion rate (add-to-cart actions)
3. Search intensity (exploration behaviour)
4. Page diversity (breadth of interest)

In [23]:
ClickstreamData

,Action,Page,Timestamp,CustomerID
0,Add to Cart,register,2020-09-13 17:06:44,1001
1,Search,login,2022-03-30 14:51:52,1001
2,Click,about,2019-11-10 05:48:48,1001
3,Add to Cart,terms,2019-05-15 10:17:44,1001
4,Add to Cart,author,2022-07-14 03:40:53,1001
...,...,...,...,...
319611,Search,terms,2020-05-25 09:12:57,13483
319612,Search,faq,2022-10-22 16:58:27,13483
319613,Search,index,2019-12-02 01:00:43,13483
319614,Add to Cart,author,2019-07-13 11:12:37,13483


In [24]:
ClickstreamData = ClickstreamData.groupby('CustomerID').agg({
    'Action': ['count', lambda x: (x == 'Add to Cart').sum(), lambda x: (x == 'Search').sum(), lambda x: (x == 'Click').sum()],
    'Page': 'nunique'
}).reset_index()
ClickstreamData.columns = ['CustomerID', 'total_actions', 'add_to_cart_count','search_count', 'click_count', 'unique_pages']
ClickstreamData['cart_conversion_rate'] = ClickstreamData['add_to_cart_count'] / (ClickstreamData['total_actions'] + 1)
ClickstreamData['search_intensity'] =  ClickstreamData['search_count'] / (ClickstreamData['total_actions'] + 1)
ClickstreamData['page_diversity'] = ClickstreamData['unique_pages'] / (ClickstreamData['total_actions'] + 1)

In [25]:
ClickstreamData

,CustomerID,total_actions,add_to_cart_count,search_count,click_count,unique_pages,cart_conversion_rate,search_intensity,page_diversity
0,1001,24,8,12,4,13,0.320000,0.480000,0.520000
1,1002,24,8,7,9,13,0.320000,0.280000,0.520000
2,1003,12,2,7,3,7,0.153846,0.538462,0.538462
3,1004,47,15,16,16,14,0.312500,0.333333,0.291667
4,1005,30,17,4,9,12,0.548387,0.129032,0.387097
...,...,...,...,...,...,...,...,...,...
12478,13479,6,4,1,1,6,0.571429,0.142857,0.857143
12479,13480,9,3,3,3,8,0.300000,0.300000,0.800000
12480,13481,26,10,11,5,9,0.370370,0.407407,0.333333
12481,13482,38,7,15,16,12,0.179487,0.384615,0.307692


#### Engagement Frequency Features

Idea: Daily users are more engaged than monthly users.

We convert frequency categories to numeric scores:

1. Daily = 30 (highest engagement)
2. Weekly = 4
3. Monthly = 1 (lowest engagement)

In [26]:
EngagementMetrics

,Logins,Frequency,CustomerID
0,19,Weekly,1001
1,9,Weekly,1002
2,19,Monthly,1003
3,4,Daily,1004
4,12,Weekly,1005
...,...,...,...
12478,22,Daily,13479
12479,25,Weekly,13480
12480,9,Monthly,13481
12481,2,Monthly,13482


In [27]:
frequency_map = {'Daily': 30, 'Weekly': 4, 'Monthly': 1}
EngagementMetrics['frequency_score'] = EngagementMetrics['Frequency'].map(frequency_map)
EngagementMetrics['engagement_score'] = EngagementMetrics['Logins'] * EngagementMetrics['frequency_score']
EngagementMetrics = EngagementMetrics[['CustomerID', 'Logins', 'frequency_score', 'engagement_score']]

In [28]:
EngagementMetrics

,CustomerID,Logins,frequency_score,engagement_score
0,1001,19,4,76
1,1002,9,4,36
2,1003,19,1,19
3,1004,4,30,120
4,1005,12,4,48
...,...,...,...,...
12478,13479,22,30,660
12479,13480,25,4,100
12480,13481,9,1,9
12481,13482,2,1,2


#### Sentiment and Feedback Features

Idea: Negative feedback and low ratings predict churn.

we calculate:

1. Average rating
2. Comment length (longer = more invested)
3. Sentiment flags (negative/positive)

In [29]:
Feedback

,Rating,Comment,CustomerID
0,1,I move baby go small big. Office institution s...,1001
1,2,Wish what bag cut life. Statement might opport...,1002
2,4,Some Democrat guess but short. Whether behind ...,1003
3,1,Yard feel never miss ask billion Congress. Fly...,1004
4,3,Ten determine unit interview challenge stock. ...,1005
...,...,...,...
12478,2,Light appear fight lawyer where star.,13479
12479,3,Yet very girl history. Thing late dream you re...,13480
12480,5,Offer particularly single degree seem sound. S...,13481
12481,5,Rest something concern likely movie. Foot in i...,13482


In [30]:
Feedback = Feedback.groupby('CustomerID').agg(
    avg_rating = ('Rating', 'first'),
    avg_comment_length = ('Comment', lambda x: x.str.len().mean())
).reset_index()
Feedback['sentiment_score'] = Feedback['avg_rating'] / 5
Feedback['is_negative'] = (Feedback['avg_rating'] <= 2).astype(int)
Feedback['is_positive'] = (Feedback['avg_rating'] >= 4).astype(int)

In [31]:
Feedback

,CustomerID,avg_rating,avg_comment_length,sentiment_score,is_negative,is_positive
0,1001,1,96.0,0.2,1,0
1,1002,2,108.0,0.4,1,0
2,1003,4,72.0,0.8,0,1
3,1004,1,78.0,0.2,1,0
4,1005,3,99.0,0.6,0,0
...,...,...,...,...,...,...
12478,13479,2,37.0,0.4,1,0
12479,13480,3,102.0,0.6,0,0
12480,13481,5,134.0,1.0,0,1
12481,13482,5,113.0,1.0,0,1


#### Marketing Response Features

Idea: Customers who ignore emails are disengaging.

We calculate:

1. Email open rate
2. Click rate
3. Click-through rate (clicks per open)
4. Overall marketing engagements

In [32]:
MarketingCommunication

,Email_Sent,Email_Opened,Email_Clicked,CustomerID
0,2019-10-17,2022-01-12,2022-11-27,1001
1,2019-10-17,2022-01-12,2022-11-27,1001
2,2019-10-17,2022-01-12,2022-11-27,1001
3,2019-10-17,2022-01-12,2022-11-27,1001
4,2019-10-17,2022-01-12,2022-11-27,1001
...,...,...,...,...
68757,2021-02-08,2021-09-18,2022-04-11,13483
68758,2021-02-08,2021-09-18,2022-04-11,13483
68759,2021-02-08,2021-09-18,2022-04-11,13483
68760,2021-02-08,2021-09-18,2022-04-11,13483


In [33]:
MarketingCommunication = MarketingCommunication.groupby('CustomerID').agg(
    emails_sent = ('Email_Sent', 'count'),
    emails_opened = ('Email_Opened', 'count'),
    emails_clicked = ('Email_Clicked', 'count')
).reset_index()
MarketingCommunication['open_rate'] = MarketingCommunication['emails_opened'] / (MarketingCommunication['emails_sent'] + 1)
MarketingCommunication['click_rate'] = MarketingCommunication['emails_clicked'] / (MarketingCommunication['emails_sent'] + 1)
MarketingCommunication['click_through_rate'] = MarketingCommunication['emails_clicked'] / (MarketingCommunication['emails_opened'] + 1)
MarketingCommunication['marketing_engagement'] = MarketingCommunication['open_rate'] * MarketingCommunication['click_rate']

In [34]:
MarketingCommunication

,CustomerID,emails_sent,emails_opened,emails_clicked,open_rate,click_rate,click_through_rate,marketing_engagement
0,1001,8,8,8,0.888889,0.888889,0.888889,0.790123
1,1002,9,9,9,0.900000,0.900000,0.900000,0.810000
2,1003,8,8,8,0.888889,0.888889,0.888889,0.790123
3,1004,10,10,10,0.909091,0.909091,0.909091,0.826446
4,1005,7,7,7,0.875000,0.875000,0.875000,0.765625
...,...,...,...,...,...,...,...,...
12478,13479,4,4,4,0.800000,0.800000,0.800000,0.640000
12479,13480,7,7,7,0.875000,0.875000,0.875000,0.765625
12480,13481,5,5,5,0.833333,0.833333,0.833333,0.694444
12481,13482,1,1,1,0.500000,0.500000,0.500000,0.250000


#### Demographics and Temporal Features

Idea: Age, segment, and account age affect churn likelihood.

We create:

1. Age groups (Young, Adult, Middle, Senior)
2. NPS categories (Detractor, Passive, Promoter)
3. Account age in days

In [37]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data['account_age_days'] = (pd.to_datetime('2023-01-01') - data['Timestamp']).dt.days
data['age_group'] = pd.cut(data['Age'], bins=[0, 25, 35, 50, 100], labels=['Young', 'Adult', 'Middle', 'Senior'])
data['nps_category'] = pd.cut(data['NPS'], bins=[-1, 6, 8, 10], labels=['Detractor', 'Passive', 'Promoter'])
demographic_features = data[['CustomerID', 'Age', 'Gender', 'Segment', 'NPS', 'account_age_days', 'age_group', 'nps_category']]

In [38]:
demographic_features

,CustomerID,Age,Gender,Segment,NPS,account_age_days,age_group,nps_category
0,1001,31,Male,Segment B,3,1069,Adult,Detractor
1,1002,66,Female,Segment C,6,1455,Senior,Detractor
2,1003,36,Female,Segment B,3,1341,Middle,Detractor
3,1004,62,Female,Segment C,1,1033,Senior,Detractor
4,1005,68,Female,Segment C,3,1366,Senior,Detractor
...,...,...,...,...,...,...,...,...
12478,13479,55,Female,Segment A,8,338,Senior,Passive
12479,13480,29,Male,Segment A,7,930,Adult,Passive
12480,13481,38,Male,Segment C,1,809,Middle,Detractor
12481,13482,26,Female,Segment A,0,920,Adult,Detractor


#### Merge All Features

In [39]:
main_df = demographic_features.merge(PurchaseHistory, on='CustomerID', how='left') \
    .merge(SubscriptionDetails, on='CustomerID', how='left') \
    .merge(ServiceInteractions, on='CustomerID', how='left') \
    .merge(PaymentHistory, on='CustomerID', how='left') \
    .merge(WebsiteUsage, on='CustomerID', how='left') \
    .merge(ClickstreamData, on='CustomerID', how='left') \
    .merge(EngagementMetrics, on='CustomerID', how='left') \
    .merge(Feedback, on='CustomerID', how='left') \
    .merge(MarketingCommunication, on='CustomerID', how='left')

main_df['ChurnLabel'] = data['ChurnLabel']

#### RFM Analysis

Recency, Frequency and Monetary Analysis

In [53]:
rfm = main_df[['CustomerID']].copy()
reference_date = main_df['Last_Interaction_Date'].max() + pd.Timedelta(days=1)
rfm['Recency'] = (reference_date - main_df['Last_Interaction_Date']).dt.days

rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F_Score'] = pd.qcut(main_df['Total_Purchase_Frequency'], 5, labels=[1,2,3,4,5])
rfm['M_Score'] = pd.qcut(main_df['Total_Purchase_Value'], 5, labels=[1,2,3,4,5])

rfm[['R_Score', 'F_Score', 'M_Score']] = (rfm[['R_Score', 'F_Score', 'M_Score']].astype(int))

rfm['rfm_score'] = rfm['R_Score'] + rfm['F_Score'] + rfm['M_Score']

rfm = rfm.drop(columns = ['Recency'])


In [54]:
rfm

,CustomerID,R_Score,F_Score,M_Score,rfm_score
0,1001,1,4,4,9
1,1002,5,1,3,9
2,1003,1,2,2,5
3,1004,3,3,2,8
4,1005,5,4,3,12
...,...,...,...,...,...
12478,13479,3,2,2,7
12479,13480,3,1,1,5
12480,13481,4,5,5,14
12481,13482,3,5,5,13


#### Encoding